### Generating team and salaries data
--------------------------------------------------------------------------
To add: description of data/data source

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [ ]:
#Teams data import

teams_df = pd.read_csv("Baseball Dataset\Teams.csv")
                          
teams_df.head()

#Limiting data to years 1985 - 2015
teams_df = teams_df[teams_df['yearID']>=1985].reset_index(drop=True)

teams_df.head()

teams_df.groupby('franchID').count()

#Limiting to franchise in full sample
    #Note: The Arizona Diamondbacks, Colorado Rockies, Flordia Marlins, and Tampa Bay Devils franchises 
    #did not exist during the full study time perion (1985-2015) and are dropped from the dataset

excluded_franchID = ['ARI','COL','FLA','TBD']

teams_df = teams_df.loc[~teams_df['franchID'].isin(excluded_franchID)]

teams_df.groupby('franchID').count()


In [ ]:
#Salaries data import

salaries_df = pd.read_csv("Baseball Dataset\Salaries.csv")
                          
salaries_df.head()



In [ ]:
#Team player salaries summary statistics
team_salary_stats = salaries_df.groupby(['yearID','teamID']).agg(total_player_salaries = ('salary', 'sum'),
                                                                  med_player_salary = ('salary','median'),
                                                                  min_player_salary = ('salary','min'),
                                                                  max_player_salary = ('salary','max'),
                                                                  salary_var = ('salary','var'))

team_salary_stats.head()

In [ ]:
#Checking # of teams per year
team_salary_stats.groupby('yearID').nunique()

#Consider restricting our sample to original 26 teams


In [ ]:
#Merging Team and Salary Data
team_salary_df = pd.merge(teams_df, team_salary_stats, on = ('yearID','teamID'))

team_salary_df.head() 

# Export the merged data into a csv
team_salary_df.to_csv("Output/team_salary_merged.csv", index=False)
    

In [ ]:
#Adding CBT thresholds and flags for 2003-2015

thresholds = pd.read_csv("Salary Thresholds/2003-2015 Salary Thresholds.csv", thousands=',')

team_salary_2003_2015 = pd.merge(team_salary_df,thresholds, left_on='yearID', right_on='Year')

team_salary_2003_2015['over_threshold'] =np.where(team_salary_2003_2015['total_player_salaries']>team_salary_2003_2015['Threshold'], 1,0)

team_salary_2003_2015['times_over_threshold'] = team_salary_2003_2015.groupby('franchID')['over_threshold'].cumsum()

team_salary_2003_2015.head()

In [ ]:
#Dataset info
team_salary_2003_2015.info() 

team_salary_2003_2015.describe()